## Athena interface

> Interface to AWS Glue and Athena

In [ ]:
#| default_exp athena

In [ ]:
#| exporti

import boto3
import json
import psycopg2
import yaml
from pathlib import Path
from io import BytesIO
import zipfile
import pandas as pd
import datetime

### Lake Interface class

In [ ]:
import sys

sys.path.append(str(Path('.').absolute().parent))

In [ ]:
#| exporti

from lakeinterface.config import ConfigManager

In [ ]:
cfgmgr = ConfigManager(profile='personal')

In [ ]:
cfgmgr.fetch_config('bankdata')

{'aurora_readdb': 'machinesp-ro.c5dgb9ysggau.us-east-1.rds.amazonaws.com',
 'aurora_secret_id': 'rds!cluster-16d6bc7f-84c2-485b-ae15-8f0afd6a2756',
 'aurora_writedb': 'machinesp-instance-1.c5dgb9ysggau.us-east-1.rds.amazonaws.com',
 'bucket': 'machinesp-datasets'}

In [ ]:
#| export



class LakeConfigNotFound(Exception):
    pass


class Athena():

    def __init__(self, profile_name=None):
        
        self.session = boto3.session.Session(profile_name=profile_name)
        
        try:
            self.config = ConfigManager(profile=profile_name).fetch_config('bankdata')
        except Exception as e:
            raise LakeConfigNotFound(e)
        
        self.athena = session.client('athena')
        self.queries = []
        self.query_results_location = f's3://{self.bucket}/{config.get("query_results_location", "athena_results")}'
        self.athena_workgroup = config.get('athena_workgroup', 'primary')
    
                
    def start_query(self, query_def, query_id):
        
        response = self.athena.start_query_execution(
            QueryString=query_def,
            QueryExecutionContext={
                'Database': 'bankdata'
            },
            WorkGroup=self.athena_workgroup,
            ResultConfiguration={"OutputLocation": self.query_results_location}
        )
        
        query_status = response['ResponseMetadata'].get('HTTPStatusCode')
        
        if query_status == 200:
            query_record = {
                'query_def': query_def,
                'query_id': query_id,
                'execution_id': response['QueryExecutionId']
            }
            try:
                query_ids = [r['query_id'] for r in self.queries]
                query_index = query_ids.index(query_id)
                self.queries[query_index] = query_record
            except:
                # query_id is for new query
                self.queries.append(query_record)
                
            response = {
                'status': 'Query Started',
                'execution_id': response['QueryExecutionId'],
                'query_id': query_id
            }
        else:
            response = response['ResponseMetadata']
            response['status'] = 'Query Failed'
        
        return response

    
    def get_query_results(self, query_id):
        query_result_gen = (
            q for q in self.queries
            if q['query_id'] == query_id
        )

        query = next(query_result_gen)
        
        if query:
            results_paginator = self.athena.get_paginator('get_query_results')
            results_iter = results_paginator.paginate(
                QueryExecutionId=query.get('execution_id'),
                PaginationConfig={
                    'PageSize': 1000
                }
            )
            
            data = []
            for rslt_page in results_iter:
                page_data = [[e.get('VarCharValue') for e in row['Data']] for row in rslt_page['ResultSet']['Rows']]
                data.append(page_data)
            
            
            return pd.DataFrame(columns=data[0], data=data[1:])
        else:
            return None



### Utilities

Functions for Common tasks

#### Unzipping from S3 to S3

In [ ]:
#| exporti

def unzip(lake_interface, source_file, destination_folder, exclude_pattern=None, include_pattern=None):
    logs = [
        '-' * 30,
        f'Copying from {source_file} to {destination_folder}'
    ]
    
        
    zip_obj = lake_interface.get_object(source_file)
    buffer = BytesIO(zip_obj["Body"].read())

    z = zipfile.ZipFile(buffer)
    file_names = z.namelist()
    filtered_files = file_names.copy()

    if include_pattern:
        filtered_files = [f for f in filtered_files if include_pattern in f]

    if exclude_pattern:
        filtered_files = [f for f in filtered_files if exclude_pattern not in f]

    for filename in filtered_files:
        file_info = z.getinfo(filename)

        lake_interface.s3.upload_fileobj(
            Fileobj=z.open(filename),
            Bucket=lake_interface.bucket,
            Key=f'{destination_folder}/{filename}'
        )

        logs.append(f'Copied {filename}')

    return logs


#### Fetching Glue Data Catalog

In [ ]:
#| exporti

def parse_column_info(cols):
    return [
        {
            'column_name': c['Name'], 
            'column_type': c['Type']
        } for c in cols if '__index_level' not in c['Name']
    ]
    
def parse_table_info(raw_table):
    return {
        'table_name': raw_table['Name'],
        'database_name': raw_table['DatabaseName'],
        'location': raw_table['StorageDescriptor']['Location'],
        'columns': parse_column_info(raw_table['StorageDescriptor']['Columns'])
    }

In [ ]:
#| export

def get_data_catalog(lake_interface, database_name):
    """
    Fetches catalog for glue database

    Parameters
    ----------
    lake_interface: LakeInterface, required
        instance of class providing access to datalake
    database_name : str, required
        Name of AWS Glue database

    Returns
    -------
    table_columns : list(str)
        List of all columns in every table in form [table_name].[column_name]

    """
    #harvest aws crawler metadata

    session = lake_interface.session
    glue = session.client('glue')

    next_token = ""
    #glue = boto3.client('glue',region_name='us-east-1')
    tables = []

    while True:
        resp = glue.get_tables(DatabaseName=database_name, NextToken=next_token)

        for tbl in resp['TableList']:
            tables.append(parse_table_info(tbl))
        next_token = resp.get('NextToken')
        
        if next_token is None:
            break

    return tables


In [ ]:
li = LakeInterface(config='machinesp')
bankdata_catalog = get_data_catalog(li, 'bankdata')

### Athena queries

In [ ]:
db_name = 'bankdata'
table_name = 'call_report_sched_ci'
query = f"SELECT COUNT(*) from {db_name}.{table_name}"

resp = li.start_query(query, 'count rows in ci')

Interface stores list of Athena queries

In [ ]:
li.queries

[{'query_def': 'SELECT COUNT(*) from bankdata.call_report_sched_ci',
  'query_id': 'count rows in ci',
  'execution_id': '9372cd85-286f-4ae1-bc1b-422789f95485'}]

The query id is used to retrieve the query results

In [ ]:
li.get_query_results('count rows in ci')

,_col0
0,107073
